# Air Pollution and the COVID-19 Pandemic

The COVID-19 pandemic has caused an unprecedented disruption in every aspect of human activity, on a global scale. Billions of people have been affected by the disease in numerous ways, and felt its impact in their daily lives. Fortunately and perhaps unexpectedly, the pandemic had a few beneficial effects as well. The various measures implemented by governments all over the world, like travel restrictions, border closures and lockdowns, have led to the decrease of air pollution in many countries. Air pollution is itself a major health issue, causing millions of deaths on a yearly basis. In this notebook we are going to explore the decrease of air pollution during the COVID-19 pandemic in various cities, using satellite data.

## Air Pollution decrease in Italy

In [1]:
from matplotlib import pyplot as plt
import cartopy.crs as ccrs
import matplotlib.colors as colors
import xarray as xr
import numpy as np
import cartopy
import matplotlib.gridspec as gridspec
from glob import iglob
from os.path import join
from functools import reduce
import pandas as pd
import harp
import cartopy.feature as cf
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.patches as mpatches
import imageio


In [8]:
product_path = 'data\L2'
export_path = 'data\L3'

input_files = sorted(list(iglob(join(product_path, '*'), recursive=True)))
print(input_files[13])
with xr.open_dataset(input_files[0]) as s5p_img:
    print(s5p_img)

data\L2\S5P_OFFL_L2__NO2____20190306T101000_20190306T115130_07222_01_010202_20190312T115330.zip
<xarray.Dataset>
Dimensions:  ()
Data variables:
    *empty*
Attributes:
    Conventions:                        CF-1.7
    institution:                        KNMI
    source:                             Sentinel 5 precursor, TROPOMI, space-...
    summary:                            TROPOMI/S5P NO2 1-Orbit L2 Swath 7x3.5km
    tracking_id:                        16818022-6a71-4ab0-a12c-bf71dee310a0
    id:                                 S5P_OFFL_L2__NO2____20190301T100306_2...
    time_reference:                     2019-03-01T00:00:00Z
    time_reference_days_since_1950:     25261
    time_reference_julian_day:          2458543.5
    time_reference_seconds_since_1970:  1551398400
    time_coverage_start:                2019-03-01T10:24:40Z
    time_coverage_end:                  2019-03-01T11:23:03Z
    time_coverage_duration:             PT3503.000S
    time_coverage_resolution:        